In [ ]:
!pip install unsloth
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [2]:
# Installar o  https://pytorch.org/get-started/locally/
# Verificar versão do cuda 
import torch
torch.cuda.is_available()

True

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [3]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


d:\Data Scientist\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [49]:
max_seq_length = 2048  # 2048 tokens max
dtype = None 
load_in_4bit = True

#https://docs.unsloth.ai/get-started/all-our-models

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit",#"unsloth/Llama-3.2-3B-Instruct-bnb-4bit", #"unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.9: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 7.995 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

### Inferencia antes do fine-tunning

In [ ]:
prompt_style = """Below you have some possible classes that you can use to classify a text is a expense description, based on that
return what do you think that best describes this expense according with classes gave to you.

### Instruction:
Consider the following classes:

Livros, Inglês , Curso , Produtos Saúde, Psicóloga, Alimentação, 
Aluguel, Internet, Celular, Ajuda Casa, Utilidade Casa, Pet, 
Higiene Pessoal, Barbearia, Esporte, Vestimenta, Restaurante, Viagem, 
Ferramentas, Eletrônico, Outros, Combustível, Estacionamento, Lavagem, 
Seguro, Impostos Auto, Financiamento, Empréstimos, Multas, Manutenção, 
Locomoção, Impostos, Necessários, Opcionais, Presentes, Planos, Assinatura, 
Doações, Exterior

Example:
### Description:
MERCADOOBAOBA
### Response:
Alimentação

### Description:
{}
### Response:
{}"""

In [ ]:
question = """ Farmácia """


FastLanguageModel.for_inference(model) 
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


Alimentação

### Description:
 Farmácia 



In [ ]:
import pandas as pd
#df = pd.read_csv('credit_card_transactions_202503111548.csv')

In [ ]:
df = pd.read_csv('training_data.csv')

In [ ]:
train_prompt_style = """ Classify the following expense description according to the classes below:

Livros, Inglês , Curso , Produtos Saúde, Psicóloga, Alimentação, 
Aluguel, Internet, Celular, Ajuda Casa, Utilidade Casa, Pet, 
Higiene Pessoal, Barbearia, Esporte, Vestimenta, Restaurante, Viagem, 
Ferramentas, Eletrônico, Outros, Combustível, Estacionamento, Lavagem, 
Seguro, Impostos Auto, Financiamento, Empréstimos, Multas, Manutenção, 
Locomoção, Impostos, Necessários, Opcionais, Presentes, Planos, Assinatura, 
Doações, Exterior

You need to respect classes listed above, never return a class that is not listed above.

description:
{}\n
classification:
{}\n
"""

In [ ]:
print(train_prompt_style)

 Classify the following expense description according to the classes below:

Livros, Inglês , Curso , Produtos Saúde, Psicóloga, Alimentação, 
Aluguel, Internet, Celular, Ajuda Casa, Utilidade Casa, Pet, 
Higiene Pessoal, Barbearia, Esporte, Vestimenta, Restaurante, Viagem, 
Ferramentas, Eletrônico, Outros, Combustível, Estacionamento, Lavagem, 
Seguro, Impostos Auto, Financiamento, Empréstimos, Multas, Manutenção, 
Locomoção, Impostos, Necessários, Opcionais, Presentes, Planos, Assinatura, 
Doações, Exterior

You need to respect classes listed above, never return a class that is not listed above.

description:
{}

classification:
{}




In [ ]:
df

,categoria,data,lancamento,valor,dt_competencia
0,Aluguel,2022-08-19,Quintoandar 05/05,45.80,2025-03-06
1,Utilidade Casa,2022-09-09,Leroy Merlin 04/04,109.73,2025-03-06
2,Planos,2022-09-11,Elaza Comercial Va04/10,290.00,2025-03-06
3,Eletrônico,2022-09-13,Aliexpress 04/06,20.87,2025-03-06
4,Eletrônico,2022-09-13,Aliexpress 04/06,20.10,2025-03-06
...,...,...,...,...,...
3404,Locomoção,NaN,Uber* Trip,3.00,2025-03-06
3405,Locomoção,NaN,Uber* Trip,17.40,2025-03-06
3406,Utilidade Casa,NaN,Amazon - Parcela 2/5,51.72,2025-03-06
3407,Locomoção,NaN,Uber* Trip,9.45,2025-03-06


In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN End of Sequence


def formatting_prompts_func(df_examples):
    texts = []
    for index, row in df_examples.iterrows():
        input = row["lancamento"]
        output = row["categoria"]
        text = train_prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
def formatting_prompts_func(examples):
    inputs = examples["lancamento"]
    outputs = examples["categoria"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = train_prompt_style.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
# Convertendo para Dataset Hugging Face
dataset = Dataset.from_pandas(df)


In [ ]:
dataset = dataset.map(formatting_prompts_func,batched=True)

Map: 100%|██████████| 624/624 [00:00<00:00, 5232.49 examples/s]


In [ ]:
dataset['text'][0]

' Classify the following expense description according to the classes below:\n\nLivros, Inglês , Curso , Produtos Saúde, Psicóloga, Alimentação, \nAluguel, Internet, Celular, Ajuda Casa, Utilidade Casa, Pet, \nHigiene Pessoal, Barbearia, Esporte, Vestimenta, Restaurante, Viagem, \nFerramentas, Eletrônico, Outros, Combustível, Estacionamento, Lavagem, \nSeguro, Impostos Auto, Financiamento, Empréstimos, Multas, Manutenção, \nLocomoção, Impostos, Necessários, Opcionais, Presentes, Planos, Assinatura, \nDoações, Exterior\n\nYou need to respect classes listed above, never return a class that is not listed above.\n\ndescription:\nCORNERSHOP\n\nclassification:\nAlimentação\n\n<|eot_id|>'

In [ ]:
#Trainning

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        #num_train_epochs = 5,
        #warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Tokenizing to ["text"]: 100%|██████████| 624/624 [00:00<00:00, 2231.00 examples/s]


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 624 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/1,827,777,536 (1.33% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.973900
20,0.372400
30,0.278900
40,0.237600
50,0.231400
60,0.237400


In [ ]:
question = """Uber trip"""


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([train_prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("classification:")[1])


In [ ]:
print(response[0])

<|begin_of_text|> Classify the following expense description according to the classes below:

Livros, Inglês, Curso, Produtos Saúde, Psicóloga, Alimentação, 
Aluguel, Internet, Celular, Ajuda Casa, Utilidade Casa, Pet, 
Higiene Pessoal, Barbearia, Esporte, Vestimenta, Restaurante, Viagem, 
Ferramentas, Eletrônico, Outros, Combustível, Estacionamento, Lavagem, 
Seguro, Impostos Auto, Financiamento, Empréstimos, Multas, Manutenção, 
Locomoção, Impostos, Necessários, Opcionais, Presentes, Planos, Assinatura, 
Doações, Exterior

You need to respect classes listed above, never return a class that is not listed above.

description:
Uber trip

classification:


classification:

Uber

<|eot_id|>
